In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm

2022-10-16 09:18:00.731424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-16 09:18:00.731479: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def moving_average(l, step):
    res = []
    for i in range(1, len(l)-step-1):
        res.append(sum(l[i:i+step]) / step)
    return res

def accuracy(preds, labels):
    preds = np.array(preds)
    labels = np.array(labels)
    acc = sum(preds == labels) / len(preds)
    return acc

## Read data:

In [5]:
data_dir = 'dataset/final_images/'
batch_size = 1
img_height = 160
img_width = 160
IMG_SIZE = (img_height, img_width)

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

train_generator = datagen.flow_from_directory(
    data_dir,
    seed=123,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    data_dir, # same directory as training data
    seed=123,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

Found 1352 images belonging to 5 classes.
Found 336 images belonging to 5 classes.


## Build model:

In [78]:
model = tf.keras.models.load_model('models/celeb5_model_ft10.tf')

In [79]:
loss, acc = model.evaluate(validation_generator)

336/336 [==============================] - 13s 34ms/step - loss: 0.5155 - accuracy: 0.9345


## Finetuned lite model:

In [6]:
with open('models/finetune_lite_model_celeb5', 'rb') as file:
    tflite_model_bin = file.read()

In [8]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_bin)
interpreter.allocate_tensors()  # Needed before execution!

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
predicted_labels = []
actual_labels = []

for i in tqdm(range(len(validation_generator)), desc='evaluting interpreter'):

    input_image, label = validation_generator[i]

    input = interpreter.get_input_details()[0]
    output = interpreter.get_output_details()[0]

    interpreter.set_tensor(input['index'], input_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output['index']).argmax().item()

    predicted_labels += [output]
    actual_labels += [label.argmax(-1)[0]]

evaluting interpreter: 100%|██████████| 336/336 [00:06<00:00, 49.37it/s]


In [11]:
accuracy(predicted_labels, actual_labels)

0.9345238095238095

## Training aware quantisized model: 

In [14]:
with open('models/finetune_lite_model_celeb5_qnt19', 'rb') as file:
    tflite_model_train_qnt_bin = file.read()

train_qnt_interpreter = tf.lite.Interpreter(model_content=tflite_model_train_qnt_bin)
train_qnt_interpreter.allocate_tensors()  # Needed before execution!

In [3]:
predicted_labels = []
actual_labels = []

for i in tqdm(range(len(validation_generator)), desc='evaluting interpreter'):

    input_image, label = validation_generator[i]

    input  = train_qnt_interpreter.get_input_details()[0]
    output = train_qnt_interpreter.get_output_details()[0]

    train_qnt_interpreter.set_tensor(input['index'], input_image)
    train_qnt_interpreter.invoke()
    output = train_qnt_interpreter.get_tensor(output['index']).argmax().item()

    predicted_labels += [output]
    actual_labels += [label.argmax(-1)[0]]

accuracy(predicted_labels, actual_labels)

NameError: name 'validation_generator' is not defined

## Post quantization:

In [7]:
model = tf.keras.models.load_model('models/celeb5_model_ft10.tf')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_quant_model = converter.convert()   

2022-10-04 10:58:02.783102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-04 10:58:02.783194: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-04 10:58:02.783237: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Nazim): /proc/driver/nvidia/version does not exist
2022-10-04 10:58:02.784344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 10:58:32.315041: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but th

INFO:tensorflow:Assets written to: /tmp/tmp3l6zt7mp/assets


2022-10-04 10:59:07.100112: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-10-04 10:59:07.100181: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-10-04 10:59:07.103760: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp3l6zt7mp
2022-10-04 10:59:07.183421: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-10-04 10:59:07.183516: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp3l6zt7mp
2022-10-04 10:59:07.443694: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-10-04 10:59:08.730472: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp3l6zt7mp
2022-10-04 10:59:09.279259: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 2175539

In [11]:
dynamic_qnt_interpreter = tf.lite.Interpreter(model_content=dynamic_quant_model)
dynamic_qnt_interpreter.allocate_tensors()

In [12]:
predicted_labels = []
actual_labels = []

for i in tqdm(range(len(validation_generator)), desc='evaluting interpreter'):

    input_image, label = validation_generator[i]

    input  = dynamic_qnt_interpreter.get_input_details()[0]
    output = dynamic_qnt_interpreter.get_output_details()[0]

    dynamic_qnt_interpreter.set_tensor(input['index'], input_image)
    dynamic_qnt_interpreter.invoke()
    output = dynamic_qnt_interpreter.get_tensor(output['index']).argmax().item()

    predicted_labels += [output]
    actual_labels += [label.argmax(-1)[0]]

accuracy(predicted_labels, actual_labels)

evaluting interpreter: 100%|██████████| 336/336 [02:50<00:00,  1.97it/s]


0.9375